In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.inspection import permutation_importance
from fairlearn.metrics import MetricFrame
from sklearn.linear_model import LogisticRegression
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference, demographic_parity_ratio 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_sample_weight
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing
import warnings


pd.set_option("display.max_columns", None)
warnings.simplefilter(action='ignore', category=FutureWarning)


c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migratin

In [2]:
df = pd.read_csv('car_insurance_claim.csv')

In [3]:
df.describe()

,ID,KIDSDRIV,AGE,HOMEKIDS,YOJ,TRAVTIME,TIF,CLM_FREQ,MVR_PTS,CAR_AGE,CLAIM_FLAG
count,1.030200e+04,10302.000000,10295.000000,10302.000000,9754.000000,10302.000000,10302.000000,10302.000000,10302.000000,9663.000000,10302.000000
mean,4.956631e+08,0.169288,44.837397,0.720443,10.474062,33.416424,5.329159,0.800718,1.710153,8.298148,0.266550
std,2.864675e+08,0.506512,8.606445,1.116323,4.108943,15.869687,4.110795,1.154079,2.159015,5.714450,0.442177
min,6.317500e+04,0.000000,16.000000,0.000000,0.000000,5.000000,1.000000,0.000000,0.000000,-3.000000,0.000000
25%,2.442869e+08,0.000000,39.000000,0.000000,9.000000,22.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,4.970043e+08,0.000000,45.000000,0.000000,11.000000,33.000000,4.000000,0.000000,1.000000,8.000000,0.000000
75%,7.394551e+08,0.000000,51.000000,1.000000,13.000000,44.000000,7.000000,2.000000,3.000000,12.000000,1.000000
max,9.999264e+08,4.000000,81.000000,5.000000,23.000000,142.000000,25.000000,5.000000,13.000000,28.000000,1.000000


In [4]:
df.head(2)

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,M,PhD,Professional,14,Private,"$14,230",11,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,M,z_High School,z_Blue Collar,22,Commercial,"$14,940",1,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban


In [5]:
a = sum(df['CLAIM_FLAG'] == False)
print(a)

7556


In [6]:
b = sum(df['CLAIM_FLAG'] == True)
print(b)

2746


In [7]:
df = df.drop(columns=['ID','BIRTH'],axis=1)
df = df.applymap(lambda x: x.replace('z_', '') if isinstance(x, str) else x)

print(df.head())

   KIDSDRIV   AGE  HOMEKIDS   YOJ   INCOME PARENT1  HOME_VAL MSTATUS GENDER  \
0         0  60.0         0  11.0  $67,349      No        $0      No      M   
1         0  43.0         0  11.0  $91,449      No  $257,252      No      M   
2         0  48.0         0  11.0  $52,881      No        $0      No      M   
3         0  35.0         1  10.0  $16,039      No  $124,191     Yes      F   
4         0  51.0         0  14.0      NaN      No  $306,251     Yes      M   

      EDUCATION    OCCUPATION  TRAVTIME     CAR_USE BLUEBOOK  TIF CAR_TYPE  \
0           PhD  Professional        14     Private  $14,230   11  Minivan   
1   High School   Blue Collar        22  Commercial  $14,940    1  Minivan   
2     Bachelors       Manager        26     Private  $21,970    1      Van   
3   High School      Clerical         5     Private   $4,010    4      SUV   
4  <High School   Blue Collar        32     Private  $15,440    7  Minivan   

  RED_CAR OLDCLAIM  CLM_FREQ REVOKED  MVR_PTS CLM_AMT  C

In [8]:
numerical = [
    'KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME',
    'HOME_VAL', 'TRAVTIME', 'BLUEBOOK', 'TIF', 'OLDCLAIM',
    'CLM_FREQ', 'MVR_PTS',  'CAR_AGE'
]

categorical = [
  'PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION',
    'OCCUPATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY'
]

df[categorical] = df[categorical].apply(lambda x: x.fillna(x.mode()[0]))

        
def clean_currency(x):
    if isinstance(x, str):
        return float(x.replace('$','').replace(',',''))
    return x

for col in ['INCOME', 'HOME_VAL', 'BLUEBOOK', 'OLDCLAIM', 'CLM_AMT']:
    df[col] = df[col].apply(clean_currency)


print("Numerical columns:", numerical)
print("Categorical columns:", categorical)

Numerical columns: ['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME', 'HOME_VAL', 'TRAVTIME', 'BLUEBOOK', 'TIF', 'OLDCLAIM', 'CLM_FREQ', 'MVR_PTS', 'CAR_AGE']
Categorical columns: ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY']


In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical),
        ("cat", Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder())
        ]), categorical)
    ]
)

In [10]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(random_state=42))
])

In [11]:
X = df.drop(columns=['CLAIM_FLAG', 'CLM_AMT'], axis=1)
y = df['CLAIM_FLAG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='roc_auc')
print(f'Cross-validation ROC-AUC: {scores.mean()} ± {scores.std()}')

Cross-validation ROC-AUC: 0.8114692760775519 ± 0.016840503120660884


In [12]:
print(df.columns)

Index(['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME', 'PARENT1', 'HOME_VAL',
       'MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'TRAVTIME', 'CAR_USE',
       'BLUEBOOK', 'TIF', 'CAR_TYPE', 'RED_CAR', 'OLDCLAIM', 'CLM_FREQ',
       'REVOKED', 'MVR_PTS', 'CLM_AMT', 'CAR_AGE', 'CLAIM_FLAG', 'URBANICITY'],
      dtype='object')


In [13]:
# # Get feature importances (absolute coefficients for logistic regression)
# importances = np.abs(pipeline..coef_[0])
# feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
# feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# print(feature_importance)

In [14]:
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      1512
           1       0.69      0.44      0.54       549

    accuracy                           0.80      2061
   macro avg       0.76      0.69      0.71      2061
weighted avg       0.79      0.80      0.78      2061

ROC-AUC Score: 0.8123235126878114


In [15]:
groups = ['GENDER', 'EDUCATION', 'MSTATUS', 'PARENT1', 'OCCUPATION', 'URBANICITY']
results = []
privileged = {}
for group in groups:
    privileged[group] = {}

In [16]:
def evaluate_accuracy(X_test, y_test, y_pred, y_pred_proba, group):
    results = []
    for value in X_test[group].unique():
        mask = X_test[group] == value
        if mask.sum() > 0:
            group_y_test = y_test[mask]
            group_y_pred = y_pred[mask]
            group_y_pred_proba = y_pred_proba[mask]
            try:
                roc_auc = roc_auc_score(group_y_test, group_y_pred_proba)
            except ValueError:
                roc_auc = float('nan')
            tn, fp, fn, tp = confusion_matrix(group_y_test, group_y_pred).ravel()

            results.append({
                "Group": group,
                "Value": value,
                "Accuracy": (group_y_test == group_y_pred).mean(),
                "ROC-AUC": roc_auc,
                "TP": tp,
                "TN": tn,
                "FP": fp,
                "FN": fn
            })

    for result in results:
        print(result)
    
    return results

    
for group in groups:
    results = evaluate_accuracy(X_test, y_test, y_pred, y_pred_proba, group)

{'Group': 'GENDER', 'Value': 'M', 'Accuracy': 0.8117770767613038, 'ROC-AUC': 0.8168753557199773, 'TP': 113, 'TN': 659, 'FP': 41, 'FN': 138}
{'Group': 'GENDER', 'Value': 'F', 'Accuracy': 0.7882882882882883, 'ROC-AUC': 0.8078115184977022, 'TP': 129, 'TN': 746, 'FP': 66, 'FN': 169}
{'Group': 'EDUCATION', 'Value': 'High School', 'Accuracy': 0.7670068027210885, 'ROC-AUC': 0.8293456071614236, 'TP': 114, 'TN': 337, 'FP': 44, 'FN': 93}
{'Group': 'EDUCATION', 'Value': 'Bachelors', 'Accuracy': 0.8271375464684015, 'ROC-AUC': 0.8290478416705626, 'TP': 53, 'TN': 392, 'FP': 22, 'FN': 71}
{'Group': 'EDUCATION', 'Value': 'Masters', 'Accuracy': 0.8259860788863109, 'ROC-AUC': 0.7397823869432166, 'TP': 16, 'TN': 340, 'FP': 6, 'FN': 69}
{'Group': 'EDUCATION', 'Value': '<High School', 'Accuracy': 0.752442996742671, 'ROC-AUC': 0.7963286713286714, 'TP': 52, 'TN': 179, 'FP': 29, 'FN': 47}
{'Group': 'EDUCATION', 'Value': 'PhD', 'Accuracy': 0.8324873096446701, 'ROC-AUC': 0.7244677011909058, 'TP': 7, 'TN': 157, 

For each group, we calculate its fairness, based on its equalized odds difference, demographic parity difference, demographic parity ratio.

Then, for each group we calculate the privileged and the unprivileged classes, based on the distance each class' value has with the most favoured value in the group.

In [17]:
def evaluate_fairness(y_true, y_pred, sensitive_features, group_name):
    eod = equalized_odds_difference(
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features
    )
    
    dpd = demographic_parity_difference(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
    )
    
    di_ratio = demographic_parity_ratio(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
    )
    
    print(f'\n group is {group_name}')
    print(f"Demographic Parity Ratio: {di_ratio:.4f}")
    print(f"Equalized Odds Difference: {eod:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    
    
    positive_rates = {}
    for group_value in sensitive_features.unique():
        mask = sensitive_features == group_value
        group_y_pred = y_pred[mask]
        positive_rate = group_y_pred.mean()
        positive_rates[group_value] = positive_rate
        print(f"Subgroup: {group_value}, Positive Prediction Rate: {positive_rate:.4f}")
    
    max_rate = max(positive_rates.values())
    min_rate = min(positive_rates.values())
    
    positive_rates = dict(sorted(positive_rates.items(), key=lambda x: x[1]))
    
    
    values = list(positive_rates.values())
    q1, q3 = np.percentile(values, [25, 75])
    iqr = q3 - q1
    threshold = iqr * 0.5 
        
        
    
    
    for group_value, rate in positive_rates.items():
        
        if rate == min_rate or (rate - min_rate <= threshold):
            print(f"--> Privileged Group: {group_value} (Positive Rate: {rate:.4f})")
            
            if 'privileged' in privileged[group_name]:
                privileged[group_name]['privileged'].append(group_value)
            else:
                privileged[group_name]['privileged'] = [group_value]

        elif rate == max_rate:
            
            print(f"--> Unprivileged Group: {group_value} (Positive Rate: {rate:.4f})")
            if 'unprivileged' in privileged[group_name]:
                privileged[group_name]['unprivileged'].append(group_value)
            else:
                privileged[group_name]['unprivileged'] = [group_value]
            
            
            
            
            
            

In [18]:
for group in groups:
    
    evaluate_fairness(y_test, y_pred, X_test[group], group)


 group is GENDER
Demographic Parity Ratio: 0.9218
Equalized Odds Difference: 0.0227
Demographic Parity Difference: 0.0137
Subgroup: M, Positive Prediction Rate: 0.1619
Subgroup: F, Positive Prediction Rate: 0.1757
--> Privileged Group: M (Positive Rate: 0.1619)
--> Unprivileged Group: F (Positive Rate: 0.1757)

 group is EDUCATION
Demographic Parity Ratio: 0.1900
Equalized Odds Difference: 0.3625
Demographic Parity Difference: 0.2177
Subgroup: High School, Positive Prediction Rate: 0.2687
Subgroup: Bachelors, Positive Prediction Rate: 0.1394
Subgroup: Masters, Positive Prediction Rate: 0.0510
Subgroup: <High School, Positive Prediction Rate: 0.2638
Subgroup: PhD, Positive Prediction Rate: 0.0660
--> Privileged Group: Masters (Positive Rate: 0.0510)
--> Privileged Group: PhD (Positive Rate: 0.0660)
--> Privileged Group: Bachelors (Positive Rate: 0.1394)
--> Unprivileged Group: High School (Positive Rate: 0.2687)

 group is MSTATUS
Demographic Parity Ratio: 0.4110
Equalized Odds Differe

Based on the above fairness metrics, we will remove gender and marital status from our groups list, for we do not deem them biased enough.

In [19]:
display(df.head(2))

,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,0,60.0,0,11.0,67349.0,No,0.0,No,M,PhD,Professional,14,Private,14230.0,11,Minivan,yes,4461.0,2,No,3,0.0,18.0,0,Highly Urban/ Urban
1,0,43.0,0,11.0,91449.0,No,257252.0,No,M,High School,Blue Collar,22,Commercial,14940.0,1,Minivan,yes,0.0,0,No,0,0.0,1.0,0,Highly Urban/ Urban


We see that the columns occupation, parent1, and occupation indicate the presence of bias in our dataset.

In [20]:
print(groups)

['GENDER', 'EDUCATION', 'MSTATUS', 'PARENT1', 'OCCUPATION', 'URBANICITY']


In [21]:
groups.remove('MSTATUS')
groups.remove('GENDER')

In [22]:
del privileged['GENDER']
del privileged['MSTATUS']
for keys,values in privileged.items():
    print(keys,values)

EDUCATION {'privileged': ['Masters', 'PhD', 'Bachelors'], 'unprivileged': ['High School']}
PARENT1 {'privileged': ['No'], 'unprivileged': ['Yes']}
OCCUPATION {'privileged': ['Manager', 'Doctor', 'Lawyer', 'Professional'], 'unprivileged': ['Blue Collar']}
URBANICITY {'privileged': ['Highly Rural/ Rural'], 'unprivileged': ['Highly Urban/ Urban']}


Let us use aif360 for mitigating bias in the education column

In [23]:

for group in groups:
    
    aif_dict = {}

    for element in privileged[group]['privileged']:    
        aif_dict[element] = 1
    
    for element in privileged[group]['unprivileged']:
        aif_dict[element] = 0


    privileged_class = [key for key, value in aif_dict.items() if value == 1]
    unprivileged_class = [key for key, value in aif_dict.items() if value == 0]

    print(f'privileged classes for group {group} are {privileged_class}')
    print(f'unprivileged classes for group {group} are {unprivileged_class}')
    
    #map column values to 0,1 s, based on whether or not the entry is privileged
    #df[group] = df[group].apply(lambda x: 1 if x in(privileged_class) else 0)




encoder = LabelEncoder()

categorical_columns = [
    'MSTATUS', 'GENDER', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'EDUCATION', 'OCCUPATION','PARENT1','URBANICITY'
]

for col in categorical_columns:
    # Fit and transform the column to encode categorical values
    df[col] = encoder.fit_transform(df[col].astype(str))  # Ensure all categories are considered by converting to string

    # Optionally, print the mapping of original values to encoded labels
    print(f"Encoded {col}: {dict(zip(encoder.classes_, range(len(encoder.classes_))))}")





# Collect all unique class values from all columns
all_unique_values = set()

# Collect unique values for each column
for col in categorical_columns:
    all_unique_values.update(df[col].astype(str).unique())

encoder.fit(sorted(all_unique_values)) 


privileged classes for group EDUCATION are ['Masters', 'PhD', 'Bachelors']
unprivileged classes for group EDUCATION are ['High School']
privileged classes for group PARENT1 are ['No']
unprivileged classes for group PARENT1 are ['Yes']
privileged classes for group OCCUPATION are ['Manager', 'Doctor', 'Lawyer', 'Professional']
unprivileged classes for group OCCUPATION are ['Blue Collar']
privileged classes for group URBANICITY are ['Highly Rural/ Rural']
unprivileged classes for group URBANICITY are ['Highly Urban/ Urban']
Encoded MSTATUS: {'No': 0, 'Yes': 1}
Encoded GENDER: {'F': 0, 'M': 1}
Encoded CAR_USE: {'Commercial': 0, 'Private': 1}
Encoded CAR_TYPE: {'Minivan': 0, 'Panel Truck': 1, 'Pickup': 2, 'SUV': 3, 'Sports Car': 4, 'Van': 5}
Encoded RED_CAR: {'no': 0, 'yes': 1}
Encoded REVOKED: {'No': 0, 'Yes': 1}
Encoded EDUCATION: {'<High School': 0, 'Bachelors': 1, 'High School': 2, 'Masters': 3, 'PhD': 4}
Encoded OCCUPATION: {'Blue Collar': 0, 'Clerical': 1, 'Doctor': 2, 'Home Maker': 3

LabelEncoder()

In [24]:
# # # Initialize a global LabelEncoder
# # encoder = LabelEncoder()

# # # Collect all unique categorical values across all columns
# # all_unique_values = set()
# # for col in df.select_dtypes(include=['object']).columns:  # Select only categorical columns
# #     all_unique_values.update(df[col].astype(str).unique())

# # # Fit the encoder on all unique values
# # encoder.fit(list(all_unique_values))

# # # Dictionary to store mappings for each column
# # encoding_mappings = {}

# # # Encode each categorical column using the global encoder
# # for col in df.select_dtypes(include=['object']).columns:
# #     # Transform the column using the global encoder
# #     df[col] = encoder.transform(df[col].astype(str))

# #     # Store the mapping for this column
# #     encoding_mappings[col] = dict(zip(encoder.classes_, range(len(encoder.classes_))))

# #     # Print the mapping for verification
# #     print(f"Encoded {col}: {encoding_mappings[col]}")

# # # Display the updated DataFrame
# # print("\nDataFrame after encoding:")
# # print(df)
# class_mapping={}
# def encode_df(df):
#     # Initialize LabelEncoder
#     encoder = LabelEncoder()

#     # List of categorical columns to encode
#     categorical_columns = [
#         'MSTATUS', 'GENDER', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'EDUCATION', 'OCCUPATION', 'URBANICITY','PARENT1'
#     ]

#     # Create a dictionary to map all unique values across all categorical columns to unique labels
#     unique_classes = set()

#     # Collect all unique class values across the categorical columns
#     for col in categorical_columns:
#         unique_classes.update(df[col].astype(str).unique())

#     # Create a mapping for all unique classes
#     class_mapping = {value: idx for idx, value in enumerate(unique_classes)}

#     # Encode each categorical column using the mapping
#     for col in categorical_columns:
#         df[col] = df[col].astype(str).map(class_mapping)  # Apply the mapping

#     # Optionally, print the class mapping for each column
#     for col in categorical_columns:
#         print(f"Encoded {col}: {dict(zip(class_mapping.keys(), class_mapping.values()))}")

#     return df

In [25]:
# encoded_df = encode_df(df)
display(df.head(2))

,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,0,60.0,0,11.0,67349.0,0,0.0,0,1,4,6,14,1,14230.0,11,0,1,4461.0,2,0,3,0.0,18.0,0,1
1,0,43.0,0,11.0,91449.0,0,257252.0,0,1,2,0,22,0,14940.0,1,0,1,0.0,0,0,0,0.0,1.0,0,1


In [26]:
for col in df.columns:
    if df[col].dtype == 'object': 
        print(col)

In [27]:
# # Print the class mapping for the 'EDUCATION' column from the global class_mapping
# education_class_mapping = {k: v for k, v in class_mapping.items() if k in df['EDUCATION'].astype(str).unique()}
# print(f"Class mapping for EDUCATION: {education_class_mapping}")


In [ ]:


# Handle missing values (if necessary)
#df.fillna(0, inplace=True)  # Fill missing values with 0 (or use another strategy)

label_name = 'CLAIM_FLAG'
favorable_classes = [0]
protected_attribute_names = ['EDUCATION', 'PARENT1', 'OCCUPATION']
privileged_classes = [[4,3], [0], [5,4,2]]  

# Optional parameters (if needed)
#categorical_features = ['MSTATUS', 'GENDER', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED']  # Specify categorical features
features_to_drop = ['id', 'address', 'CLM_AMT']  # Columns to drop (if any)
na_values = ['NA', '?', '']  # Handle missing values

# Now, create the StandardDataset instance
dataset = StandardDataset(
    df.drop('CLM_AMT',axis=1),
    label_name=label_name,
    favorable_classes=favorable_classes,
    protected_attribute_names=protected_attribute_names,
    privileged_classes=privileged_classes,
    # categorical_features=categorical_columns,  # Pass all categorical columns
    # features_to_drop=features_to_drop,
    # na_values=na_values
)



# Thedataset object is now processed and ready to be used for further analysis
#display(dataset)


KeyError: "['CLAIM_AMT'] not found in axis"

In [29]:
display(df.head(10))

,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,0,60.0,0,11.0,67349.0,0,0.0,0,1,4,6,14,1,14230.0,11,0,1,4461.0,2,0,3,0.0,18.0,0,1
1,0,43.0,0,11.0,91449.0,0,257252.0,0,1,2,0,22,0,14940.0,1,0,1,0.0,0,0,0,0.0,1.0,0,1
2,0,48.0,0,11.0,52881.0,0,0.0,0,1,1,5,26,1,21970.0,1,5,1,0.0,0,0,2,0.0,10.0,0,1
3,0,35.0,1,10.0,16039.0,0,124191.0,1,0,2,1,5,1,4010.0,4,3,0,38690.0,2,0,3,0.0,10.0,0,1
4,0,51.0,0,14.0,NaN,0,306251.0,1,1,0,0,32,1,15440.0,7,0,1,0.0,0,0,0,0.0,6.0,0,1
5,0,50.0,0,NaN,114986.0,0,243925.0,1,0,4,2,36,1,18000.0,1,3,0,19217.0,2,1,3,0.0,17.0,0,1
6,0,34.0,1,12.0,125301.0,1,0.0,0,0,1,0,46,0,17430.0,1,4,0,0.0,0,0,0,2946.0,7.0,1,1
7,0,54.0,0,NaN,18755.0,0,NaN,1,0,0,0,33,1,8780.0,1,3,0,0.0,0,0,0,0.0,1.0,0,1
8,1,40.0,1,11.0,50815.0,1,0.0,0,1,2,5,21,1,18930.0,6,0,0,3295.0,1,0,2,6477.0,1.0,1,1
9,0,44.0,2,12.0,43486.0,1,0.0,0,0,2,0,30,0,5900.0,10,3,0,0.0,0,0,0,0.0,10.0,0,0


In [30]:
print(groups)

['EDUCATION', 'PARENT1', 'OCCUPATION', 'URBANICITY']


In [31]:
unprivileged_groups = [{'EDUCATION': 2}]
privileged_groups = [{'EDUCATION': 4}, {'EDUCATION': 3}, {'EDUCATION': 1}] 

unprivileged_groups.append({'PARENT1': 1})
privileged_groups.append({'PARENT1': 0})

unprivileged_groups.append({'OCCUPATION': 7})
privileged_groups.append({'OCCUPATION': 5})
privileged_groups.append({'OCCUPATION' : 4}) 
privileged_groups.append({'OCCUPATION' : 2}) 



reweighing = Reweighing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)
reweighed_data = reweighing.fit_transform(dataset)
X_train, X_test, y_train, y_test = train_test_split(
    reweighed_data.features, reweighed_data.labels.ravel(), test_size=0.3, random_state=42
)

X_test = pd.DataFrame(X_test, columns=X.columns)

model = LogisticRegression(penalty='l2', C=0.1, random_state=42, max_iter=1000)
model.fit(X_train, y_train)


scores = cross_val_score(model, X_train, y_train, cv=10, scoring='roc_auc')
print(f'Cross-validation ROC-AUC: {scores.mean()} ± {scores.std()}')


y_pred = model.predict(X_test)

dataset_test = dataset.copy()  # Original test dataset (without reweighing)
dataset_test.features = X_test
dataset_test.labels = y_test

# For reweighed data, we use the predictions made by the model 
reweighed_dataset_test = dataset.copy()
reweighed_dataset_test.features = X_test
reweighed_dataset_test.labels = y_pred





y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class




# # Step 6: Evaluate fairness using ClassificationMetric
# metric_original = ClassificationMetric(dataset_test, dataset_test.copy())  # Original data without reweighing
# metric_reweighed = ClassificationMetric(reweighed_dataset_test, reweighed_dataset_test.copy())  # Reweighed data

# # Example of fairness metrics (you can print more specific metrics based on your needs)
# print(f"Accuracy on test set: {accuracy_score(y_test, y_pred)}")
# print(f"Disparate Impact: {metric_reweighed.disparate_impact()}")
# print(f"Equal Opportunity Difference: {metric_reweighed.equal_opportunity_difference()}")
# print(f"Average Odds Difference: {metric_reweighed.average_odds_difference()}")


ValueError: Shape of passed values is (2449, 24), indices imply (2449, 23)

In [ ]:
groups.remove('URBANICITY')

for group in groups:
    #evaluate_fairness(y_test, y_pred, X_test[group], group)
    evaluate_accuracy(X_test, y_test, y_pred, y_pred_proba, group)

{'Group': 'EDUCATION', 'Value': 3.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 97, 'TN': 392, 'FP': 0, 'FN': 0}
{'Group': 'EDUCATION', 'Value': 1.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 180, 'TN': 508, 'FP': 0, 'FN': 0}
{'Group': 'EDUCATION', 'Value': 0.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 126, 'TN': 258, 'FP': 0, 'FN': 0}
{'Group': 'EDUCATION', 'Value': 2.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 248, 'TN': 437, 'FP': 0, 'FN': 0}
{'Group': 'EDUCATION', 'Value': 4.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 32, 'TN': 171, 'FP': 0, 'FN': 0}
{'Group': 'PARENT1', 'Value': 0.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 525, 'TN': 1604, 'FP': 0, 'FN': 0}
{'Group': 'PARENT1', 'Value': 1.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 158, 'TN': 162, 'FP': 0, 'FN': 0}
{'Group': 'OCCUPATION', 'Value': 5.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 36, 'TN': 273, 'FP': 0, 'FN': 0}
{'Group': 'OCCUPATION', 'Value': 6.0, 'Accuracy': 1.0, 'ROC-AUC': 1.0, 'TP': 78, 'TN': 241, 'FP': 0, 'FN': 0}
{'Group': 'OCCU

In [ ]:
# Get feature importances (absolute coefficients for logistic regression)
importances = np.abs(model.coef_[0])
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

print(feature_importance)

       Feature  Importance
21     CLM_AMT    0.364679
1          AGE    0.148423
10  OCCUPATION    0.030812
3          YOJ    0.029755
11    TRAVTIME    0.028266
14         TIF    0.025717
22     CAR_AGE    0.019876
20     MVR_PTS    0.012253
15    CAR_TYPE    0.008811
2     HOMEKIDS    0.008655
18    CLM_FREQ    0.007227
12     CAR_USE    0.006339
9    EDUCATION    0.003355
16     RED_CAR    0.001970
5      PARENT1    0.001388
0     KIDSDRIV    0.001365
8       GENDER    0.000831
23  URBANICITY    0.000794
7      MSTATUS    0.000655
19     REVOKED    0.000641
13    BLUEBOOK    0.000365
4       INCOME    0.000008
17    OLDCLAIM    0.000005
6     HOME_VAL    0.000002
